In [1]:
# Web scraper for KanView website that provides payroll data for the state of Kansas
# Selenium is used to handle the javascript on a website

#Importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os     #to manipulate files



In [81]:
chrome_path = r'C:\Users\Abraham\Downloads\Apps\chromedriver'
# prepend it with r because of the backslash characters

driver = webdriver.Chrome(chrome_path) # This will open up a chrome window

page = 2557
url = "https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p" + str(page)

driver.get(url)  # Open that website


In [82]:
userlist = []
datelist = []
postlist = []
quotelist = []
while len(postlist)<5000:
    #User
    users = driver.find_elements_by_class_name('Username')
    for user in users:
        userlist.append(user.text)
    
    #Date
    dates = driver.find_elements_by_class_name('DateCreated')
    for date in dates:
        datelist.append(date.text)
    
    # This is the list of all the comments
    posts = driver.find_elements_by_class_name('Item-Body')
    for post in posts:
        postlist.append(post.text)

    # This is the list of the blockquotes that you don't want
    quotes = driver.find_elements_by_class_name('Quote')
    for quote in quotes:
        quotelist.append(quote.text)
        
    page = page - 1
    url = "https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p" + str(page)

    driver.get(url)
    


In [84]:
# Now you have to delete quotes from the posts
for j in range(0,len(quotelist)):
    for i in range(0,len(postlist)):
        if (quotelist[j] in postlist[i]):
            postlist[i]=postlist[i].replace(quotelist[j],'')   

In [83]:
len(postlist)

5007

In [85]:
df_raw = pd.DataFrame(datelist)
df_raw['Users'] = userlist
df_raw['Text'] = postlist
df_raw.columns = ['Date','User','Text']

In [86]:
df_raw

,Date,User,Text
0,September 26,snakeweasel,\nI went and checked and yes the 59k one is an...
1,September 26,snakeweasel,\nI would have told them that I used to but I ...
2,September 26,bwia,"\nNo, it can't be otherwise the transactions w..."
3,September 26,Mr_Shiftright,stickguy said:\nfunction is not reliability. S...
4,September 26,imidazol97,\nshow previous quotes\n\n\n\nI would have tol...
5,September 26,oldfarmer50,driver100 said:\nI hear lots of Honda and Toyo...
6,September 26,ab348,"I had a Volvo (well, Dad's, but I was the only..."
7,September 26,henryn,bwia said:\nshow previous quotes\n\n\n\n\n\nDo...
8,September 26,ab348,"Back on the TV topic, I got to watch New Amste..."
9,September 26,PF_Flyer,"So, we are SO attached to our vehicles, it's a..."


In [88]:
postfile_name = r'C:\Users\Abraham\MSBA\Text\HW2\postfile.csv'
df_raw.to_csv(postfile_name)